In [ ]:
import logging
import os
import pandas as pd
import src.constants as ct
import src.tests.testconfig as tc

from src.parsedata import PDFParser
from src.initialize import Logger
from src.capitalgains import CapitalGains
from src.utils import rename_mf_scheme


pd.options.display.max_columns = 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
!python --version

In [ ]:
logger = Logger()
logger.initilaze_logging('no')   
logging.info(f'{"*" * 50}Start Of Process{"*" * 50}')




## Pre-Requisite: CAS statement
The first step is to get the [consolidated account statement](https://www.camsonline.com/Investors/Statements/Consolidated-Account-Statement) from CAMS in PDF Format. Make sure that you select the option as highlighted in yellow.

 <!-- ![screenshot](https://github.com/ajayrawatsap/mf_data/blob/master/data/assets/cams.PNG) -->

 <img src="data/assets/cams.PNG" width=800 height=400 />




## Validate if CAS file exists

Rename your CAS statement from CAMS to *cas.pdf* and place it at root directory. It should be in same directory as this notebook demo.ipynb.
This applies to both local as well as cloud VM enviornment.

In [ ]:
import os.path
file_name = 'cas.pdf'
if not os.path.isfile(file_name):
    raise ValueError("file cas.pdf not found in root folder: Hint: Rename your CAS statement file to cas.pdf and save in root directory ")
else: 
    print(f'File {file_name} Found: OK')

##  Password Input

The password is same as what you set up when requesting the statement from CAMS website.

In [ ]:
from getpass import getpass
# password = getpass(prompt='Input your password for cas.pdf: ')
password = tc.password

## Read Contents of PDF

In [ ]:
%%time
dp = PDFParser()
    
mf_trans_df, mf_hdr_df = dp.parse_mf_data(file_name, password)
print('Parsing CAS statement Data Please wait:', end = " ")


## Calculate LTCG and STCG
By defaut Target LTCG amount is set to INR 100,000. IF you want to set it to different value (e.g.. INR 50,000) change the code in cell below and then run all cells again

target_ltcg = 50000

In [ ]:
print('Calculating LTCG and Units to Sell: ', end = " ")

target_ltcg = 100000
cg = CapitalGains(mf_hdr_df, mf_trans_df)
cg.prepare_final_data(target_ltcg)
print("Done")

## Display Header Data

<BR>The Header and Transaction Data gets saved in data/output directory of Cloud VM or Local Machine as csv files. For Cloud VM Select the checkbox againt each file and download it to your desktops

In [ ]:
from src.mfdisplay import get_hdr_data_to_display

hdr_df = get_hdr_data_to_display(cg.output_hdr_df)
hdr_df.head()

## Display Transaction Data for Single MF scheme

By default this will show transaction for a random Scheme. You can execute the cell multple times to see results for a diffrent scheme

<br>To view transaction for a specific scheme (e.g. DSP Small Cap Fund - Direct Plan - Growth)  replace  the code in cell to:-
<br> get_trans_for_single_scheme(trans_df,   scheme_name= 'DSP Small Cap Fund - Direct Plan - Growth')

Place cursor on code cell below Hit SHIFT + ENTER to see the results 


#### The *cumil_ltcg* and *cumil_units* columns are useful to know what units to sell for target ltcg.   Sell *cumil_units* for a LTCG of *cumil_ltcg*

In [ ]:
from src.mfdisplay import get_trans_data_to_display, get_trans_for_single_scheme
trans_df = get_trans_data_to_display(cg.output_trans_df)

get_trans_for_single_scheme(trans_df, scheme_name= None)


In [ ]:


agg = {    'Invested Amount': pd.NamedAgg(column= 'invested_amt', aggfunc = 'sum'),
           'Current Amount': pd.NamedAgg(column= 'current_amt', aggfunc= 'sum'),
           'Total LTCG': pd.NamedAgg(column = 'ltcg', aggfunc= 'sum'),
           'Total STCG': pd.NamedAgg(column= 'stcg', aggfunc= 'sum')
        }


mf_totals_grp = cg.mf_hdr_df.groupby('type').agg(**agg).reset_index() 
mf_totals_grp['Total Gain']  = mf_totals_grp['Current Amount'] - mf_totals_grp['Invested Amount']
mf_totals_grp['Gain Percent'] = (mf_totals_grp['Total Gain'] /  mf_totals_grp['Invested Amount']) * 100


columns = mf_totals_grp.columns.tolist()
columns.remove('type') 

mf_totals_grp[columns] = mf_totals_grp[columns].astype(float).round(2)
total_invested_amt = mf_totals_grp['Invested Amount'].sum()
total_current_amt = mf_totals_grp['Current Amount'].sum()
total_gain = mf_totals_grp['Total Gain'].sum()
perc_gain = round((total_gain / total_invested_amt) * 100, 2)
mf_totals_grp

In [ ]:
total_invested_amt, total_current_amt , total_gain, perc_gain

### Portfolio Summary

In [ ]:
from plotly import graph_objects as go

total_amounts = [total_invested_amt, total_current_amt , total_gain]
amount_names = ['Invested Amount','Current Amount', 'Gain/Loss']
fig = go.Figure( data = [

                         go.Bar( name = f"Current Amount ({ct.RUPEEE_SYMBOL})",  x =  total_amounts ,  y=  amount_names, orientation = 'h',
                                 text =  total_amounts , textposition='outside', texttemplate='%{text:.2s}'
                                 ),
                         
                        
                        ])

fig.update_layout( 
    
                #   barmode='group',
                   height=350,
                   title = 'Portfolio Summary'
                 )
fig.show()                       

##  Equiy vs Debt Aggregate Data

In [ ]:
from plotly.subplots import make_subplots
from plotly import graph_objects as go




totals_df = mf_totals_grp
mf_types = totals_df.type.to_list()


fig = make_subplots(rows=2, cols=2, 
                    specs=[
                          [{"type": "bar"}, {"type": "bar"}],
                          [{"type": "pie"}, {"type": "bar"}]
                          ],
                    subplot_titles=("Portfolio Amounts", "Capital Gains (LTCG/STCG)", " ", "Gain % (Absolute)"))

# Subplot1: Invested Anount, Current Amount, Gain
invested_amount = totals_df['Invested Amount']
current_amount = totals_df['Current Amount']
gain =  totals_df['Total Gain']


fig.add_trace(
     go.Bar(name= f"Current Amount ({ct.RUPEEE_SYMBOL})",x = current_amount, y=mf_types, orientation = 'h',  legendgroup = '1',
                                    text = current_amount, textposition='outside', texttemplate='%{text:.2s}'),
    row=1,
    col=1,
   
)


fig.add_trace(
     go.Bar(name=  f"Invested Amount ({ct.RUPEEE_SYMBOL})", x= invested_amount, y= mf_types, orientation = 'h',  legendgroup = '1',
                                 text = invested_amount, textposition='outside', texttemplate='%{text:.2s}' ),
    
     row = 1,
     col = 1,
  
)


fig.add_trace(
     
     go.Bar(name=  f"Gain/Loss ({ct.RUPEEE_SYMBOL})", x= gain, y= mf_types, orientation = 'h',  legendgroup = '1',
                                 text = gain, textposition='outside', texttemplate='%{text:.2s}' ),
     row = 1,
     col = 1,
   
)


# Subplot2:LTCG and STCG
ltcg = totals_df['Total LTCG']
stcg = totals_df['Total STCG']

fig.add_trace(
     
     go.Bar(name= f"Total LTCG ({ct.RUPEEE_SYMBOL})",x = ltcg, y=mf_types, orientation = 'h',  legendgroup = '2',
                                    text = ltcg, textposition='outside', texttemplate='%{text:.2s}'),
     row = 1,
     col = 2,
     

)

fig.add_trace(    
     
     go.Bar(name=  f"Total STCG ({ct.RUPEEE_SYMBOL})", x= stcg, y= mf_types, orientation = 'h', legendgroup = '2',
                                 text = stcg, textposition='outside', texttemplate='%{text:.2s}' ),
     
     row = 1,
     col = 2,

)

#Subplot3: Debt Vs Equity Ratio
fig.add_trace(

            go.Pie(labels = mf_types, 
                    values = current_amount,
                    legendgroup = '3',
                    textinfo='label+percent',
                    showlegend=False,
                    insidetextorientation='radial'),

            row = 2,
            col = 1,

           )




#Subplot4; Show Gain Percent
fig.add_trace(
     
     go.Bar(name= f"Gain Percent",x = totals_df['Gain Percent'], y=mf_types, orientation = 'h',  legendgroup = '4', showlegend=False,
                                    text = totals_df['Gain Percent'], textposition='outside', texttemplate='%{text:.2s}'),
     row = 2,
     col = 2,
     

)

fig.update_layout(height=700,  legend_tracegroupgap = 50)
fig.show()

## Plot Mutual Funds Scheme Amounts

In [ ]:
import plotly.graph_objects as go
hdr_df = cg.mf_hdr_df
# Show graph with sorted order of current amount
hdr_df = hdr_df.sort_values(by = ['current_amt'], ascending= False)

#Shorten the MF names 
scheme_names = [ name.split('Fund')[0].strip() for name in hdr_df.scheme_name.to_list()]
gain_loss =  hdr_df.current_amt.values - hdr_df.invested_amt.values
# gain_loss =  hdr_df.amount.values- hdr_df.current_amt.values 
fig = go.Figure( data = [

                         go.Bar( name = f"Current Amount ({ct.RUPEEE_SYMBOL})",  x =  scheme_names, y= hdr_df.current_amt.values, 
                                 text = hdr_df.current_amt.values, textposition='outside', texttemplate='%{text:.2s}'
                                 ),
                         go.Bar( name = f"Invested Amount ({ct.RUPEEE_SYMBOL})", x = scheme_names, y= hdr_df.invested_amt.values,
                                  text = hdr_df.invested_amt.values, textposition='outside', texttemplate='%{text:.2s}'),
                       
                                # text=y, textposition='auto'),
                         go.Bar( name = f"Gain/Loss ({ct.RUPEEE_SYMBOL})",  x =  scheme_names, y= gain_loss, 
                                 text = gain_loss, textposition='outside', texttemplate='%{text:.2s}')
                                # text=y,textposition='auto' )
                        
                        ])

fig.update_layout( barmode='group',
                   height=600,
                   title = 'Mutual Funds Scheme Valuation'
                 )
fig.show()                       

## Plot Unrealised LTCG and STCG

In [ ]:

hdr_df = hdr_df.sort_values(by = ['ltcg'], ascending= False)
scheme_names = [ name.split('Fund')[0].strip() for name in hdr_df.scheme_name.to_list()]

fig = go.Figure( data = [

                         go.Bar( name = f"LTCG ({ct.RUPEEE_SYMBOL})",  x =  scheme_names, y= hdr_df.ltcg.values, 
                                 text =   hdr_df.ltcg.values, textposition='outside', texttemplate='%{text:.2s}'
                                 ),

                         go.Bar( name = f"STCG ({ct.RUPEEE_SYMBOL})", x = scheme_names, y= hdr_df.stcg.values,
                                  text = hdr_df.stcg.values, textposition='outside', texttemplate='%{text:.2s}'),
                       
                        ])

fig.update_layout( barmode='group',
                   height=600,
                #    width = 800,
                   title = 'Unrealised LTCG and STCG'
                 )
fig.show()                 

## Plot Mutual Funds Allocation by Scheme

In [ ]:
import plotly.graph_objects as go

hdr_df = hdr_df.sort_values(by = ['current_amt'], ascending= False)
scheme_names = [ name.split('Fund')[0].strip() for name in hdr_df.scheme_name.to_list()]
fig = go.Figure(data = [go.Pie(labels = scheme_names, 
                               values = hdr_df.current_amt,
                               textinfo='label+percent',
                               insidetextorientation='radial',
                               
                               )])

fig.update_layout(
                   title = 'Mutual Funds Allocation'
                    )                              
fig.show()

In [ ]:

logging.info(f'{"*" * 50}End Of Process{"*" * 50}\n') 